In [1]:
import json
import os
from tqdm import tqdm
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

# ========== CONFIG ==========
LAW_PATH = "alqac25_law.json"
QA_PATH = "alqac25_private_test_Task_1.json"  # Đánh giá trên bộ test
TOP_K = 1  # Số documents top-k sau khi rerank
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# ============================

def load_laws(law_path):
    with open(law_path, encoding="utf-8") as f:
        laws = json.load(f)
    docs = []
    for law in laws:
        law_name = law["id"]
        for article in law["articles"]:
            docs.append({
                "law_id": law_name,
                "article_id": article["id"],
                "text": article["text"]
            })
    return docs

def load_questions(qa_path):
    with open(qa_path, encoding="utf-8") as f:
        return json.load(f)

def build_bm25(docs):
    corpus = [doc["text"] for doc in docs]
    tokenized_corpus = [text.split() for text in corpus]
    bm25 = BM25Okapi(tokenized_corpus)
    return bm25

def build_dense_index(docs, embedder):
    corpus = [doc["text"] for doc in docs]
    embeddings = embedder.encode(corpus, show_progress_bar=True, convert_to_numpy=True, batch_size=64)
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)
    # Lưu
    np.save("law_embeds.npy", embeddings)
    faiss.write_index(index, "faiss.index")
    return index, embeddings

def hybrid_retrieve(question, bm25, docs, embedder, dense_index, dense_corpus_embeds, top_k=3, alpha=0.5):
    # BM25: lấy 10 tài liệu
    bm25_scores = bm25.get_scores(question.split())
    bm25_top_idx = np.argsort(bm25_scores)[::-1][:5]  # Lấy top 10 từ BM25
    
    # Rerank bằng SentenceTransformer
    bm25_docs = [docs[idx] for idx in bm25_top_idx]
    bm25_texts = [doc["text"] for doc in bm25_docs]
    
    # Encode question và documents
    question_emb = embedder.encode([question], convert_to_numpy=True)
    doc_embs = embedder.encode(bm25_texts, convert_to_numpy=True)
    
    # Tính cosine similarity
    similarities = np.dot(doc_embs, question_emb.T).flatten()
    
    # Sắp xếp theo similarity và lấy top 2
    top_indices = np.argsort(similarities)[::-1][:top_k]
    
    # Trả về top 2 documents
    results = [bm25_docs[idx] for idx in top_indices]
    return results

def build_prompt(article_content, question, prompt_type="Đúng/Sai"):
    if prompt_type == "Đúng/Sai":
        prompt = (
            "Dựa nội dung sau đây, hãy xác định câu sau là Đúng hay Sai.\n\n"
            "Lưu ý: bắt buộc chỉ trả lời Đúng hay Sai không cần giải thích gì thêm.\n\n"
            "Ví dụ: câu hỏi là Hội đồng trọng tài có thể không dùng pháp luật Việt Nam để giải quyết tranh chấp, đúng hay sai?\n\n"
            "Thì đáp án: Đúng\n\n"
            f"Câu hỏi: {question}\n\n{article_content}\n\n"
            "Đáp án: "
        )
    elif prompt_type == "Trắc nghiệm":
        prompt = (
            "Dựa vào nội dung sau đây, hãy chọn đáp án đúng.\n\n"
            "Lưu ý: bắt buộc chỉ cần trả lời bằng một trong các lựa chọn A, B, C, hoặc D không cần giải thích gì thêm.\n\n"
            "Ví dụ: câu hỏi là Người xem dưới 16 tuổi được xem phim có nội dung thuộc phân loại nào sau đây? A. T18 (18+), B. T16 (16+), C. C, D. K\n\n"
            "Thì đáp án: D\n\n"
            f"Câu hỏi: {question}\n\n{article_content}\n\n"
            "Đáp án: "
        )
    elif prompt_type == "Tự luận":
        prompt = (
            "Dựa nội dung sau đây, hãy trả lời ngắn gọn, không giải thích gì thêm\n\n"
            "Ví dụ: câu hỏi là Hành vi nào liên quan đến phiên dịch được coi là hành vi cản trở thu thập xác minh chứng cớ của tòa án?\n\n"
            "Thì đáp án là Dịch sai sự thật\n\n"
            f"Câu hỏi: {question}\n\n{article_content}\n\n"
            "Đáp án: "
        )
    else:
        raise ValueError("prompt_type không hợp lệ")
    return prompt

def find_relevant_articles_for_questions(questions, docs, bm25, embedder, dense_index, dense_corpus_embeds, top_k=3):
    results = []
    for qa in tqdm(questions, desc="Tìm tài liệu liên quan"):
        question = qa["text"]
        top_docs = hybrid_retrieve(question, bm25, docs, embedder, dense_index, dense_corpus_embeds, top_k=top_k)
        relevant_articles = [
            {"law_id": doc["law_id"], "article_id": doc["article_id"]} for doc in top_docs
        ]
        results.append({
            "question_id": qa.get("question_id"),
            "text": question,
            "relevant_articles": relevant_articles
        })
    return results






d:\New folder\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\New folder\.venv\Lib\site-packages\transformers\utils\hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [ ]:

print("Tải dữ liệu...")
docs = load_laws(LAW_PATH)
questions = load_questions(QA_PATH)  # Đánh giá toàn bộ bộ test

print("Khởi tạo BM25...")
bm25 = build_bm25(docs)

print("Tải mô hình embedding ...")
embedder = SentenceTransformer("AITeamVN/Vietnamese_Embedding")

print("Xây dựng FAISS dense index...")
dense_index, dense_corpus_embeds = build_dense_index(docs, embedder)
    # Chỉ thực hiện tìm và lưu relevant_articles cho từng câu hỏi
questions_with_rels = find_relevant_articles_for_questions(questions, docs, bm25, embedder, dense_index, dense_corpus_embeds, top_k=3)


Tải dữ liệu...
Khởi tạo BM25...
Tải mô hình embedding truro7/vn-law-embedding...


Invalid model-index. Not loading eval results into CardData.
Invalid model-index. Not loading eval results into CardData.


Xây dựng FAISS dense index...


Batches:   2%|▏         | 1/53 [00:28<24:39, 28.46s/it]


KeyboardInterrupt: 

In [ ]:
with open("questions_with_relevant_articles.json", "w", encoding="utf-8") as f:
    json.dump(questions_with_rels, f, ensure_ascii=False, indent=2)
print("\nĐã lưu danh sách relevant_articles cho từng câu hỏi vào questions_with_relevant_articles.json")